In [1]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
from shapely.geometry import Point
import os
import glob
import re

In [18]:
# path to result folder
result_folder = 'C:\\code\\DEGDB\\degdb_utils\\playground\\result'
# path to locations
subject_path = 'C:\\code\\DEGDB\\degdb_utils\\playground\\espon\\GeoLocation.csv'
# path to data
folder_path = 'C:\\code\\DEGDB\\degdb_utils\\data\\espon'

In [15]:
# load subjects csv
sub_df = pd.read_csv(subject_path)
# Convert the 'geometry' column to actual geometrical data using shapely
# Create a geometry column from the latitude and longitude columns
#sub_df['geometry'] = [Point(xy) for xy in zip(sub_df['Longitude'], sub_df['Latitude'])]
sub_df['geometry'] = [Point(xy) for xy in zip(sub_df['longitude'], sub_df['latitude'])]

# Convert the pandas DataFrame to a GeoPandas GeoDataFrame
sub_df = gpd.GeoDataFrame(sub_df, geometry='geometry')
sub_df = sub_df.drop(['latitude','longitude'],axis = 1)#'City','Dist_Centre_km'],axis = 1)
#sub_df = sub_df.drop(['Latitude','Longitude','City','Dist_Centre_km'],axis = 1)
# Set the current coordinate reference system (CRS)
sub_df.set_crs(epsg=4326, inplace=True)

#print(data_gdf.head(2))
print(sub_df.head(2))

     PSC2                   geometry
0    1274  POINT (-1.26000 52.91000)
1  215284  POINT (-1.17000 53.09000)


In [16]:
# Use glob to find all CSV files in subfolders only
csv_files = glob.glob(os.path.join(folder_path, '**', '*.csv'), recursive=True)

# Filter out the CSV files that are in the main folder
espon_data_files = [file for file in csv_files if os.path.dirname(file) != folder_path]

# Print the list of CSV files in subfolders
for file in espon_data_files:
    print(file)
    break

C:\code\DEGDB\degdb_utils\data\espon\4.0_patents\Taxonomy_of_4.0_inventing_regions.csv


In [20]:
def sanitize_filename(filename):
    # Replace each whitespace with an underscore
    filename = filename.replace(' ', '_')
    # Remove characters not allowed in file names
    filename = re.sub(r'[<>:"/\\|?*]', '', filename)
    return filename

# load geometries
geometries_path = os.path.join(folder_path, 'geometry.csv')
# Load the CSV file into a pandas DataFrame
gdf = pd.read_csv(geometries_path)

# Convert the 'geometry' column to actual geometrical data using shapely
gdf['geometry'] = gdf['geometry'].apply(wkt.loads)

# Convert the pandas DataFrame to a GeoPandas GeoDataFrame
gdf = gpd.GeoDataFrame(gdf, geometry='geometry')

# Set the current coordinate reference system (CRS)
gdf.set_crs(epsg=4326, inplace=True)

i = 1
for file in espon_data_files:
    try:
        # load dataOI
        df = pd.read_csv(file)
        # Merging data and geometries on the 'tunit_code' column
        data_gdf = pd.merge(gdf, df, on='tunit_code')
        # Select columns to keep
        columns_to_keep = ['PSC2','name','level'] + [col for col in data_gdf.columns if col.startswith('y_')]
        # Perform spatial join
        joined_gdf = gpd.sjoin(sub_df, data_gdf, how='inner', predicate='within')
        joined_gdf = joined_gdf[columns_to_keep]
        
        csv_file_name = joined_gdf.iloc[0, joined_gdf.columns.get_loc('name')]
        csv_file_name = os.path.join(result_folder, sanitize_filename(csv_file_name)+'.csv')
        joined_gdf.to_csv(csv_file_name, index=False)
        print('saved: ' + csv_file_name)
    except:
        print('no result table or empty raw data file')
        print(file)
    i = 1+i
    if 0==i%100:
        print(i)

print(print('done'))

saved: C:\code\DEGDB\degdb_utils\playground\result\Taxonomy_of_4.0_inventing_regions.csv
saved: C:\code\DEGDB\degdb_utils\playground\result\4.0_patents_-_by_sector_-_in_smart_energy_technologies.csv
saved: C:\code\DEGDB\degdb_utils\playground\result\4.0_patents_-_by_sector_-_in_smart_transport_technologies.csv
saved: C:\code\DEGDB\degdb_utils\playground\result\Ageing_index_-_female_population.csv
saved: C:\code\DEGDB\degdb_utils\playground\result\Ageing_index_-_male_population.csv
no result table or empty raw data file
C:\code\DEGDB\degdb_utils\data\espon\Aggregate_impact_of_climate_change_on_regions\Potential_cultural_impact_of_climate_change.csv
no result table or empty raw data file
C:\code\DEGDB\degdb_utils\data\espon\Aggregate_impact_of_climate_change_on_regions\Potential_economic_impact_of_climate_change.csv
no result table or empty raw data file
C:\code\DEGDB\degdb_utils\data\espon\Aggregate_impact_of_climate_change_on_regions\Potential_environmental_impact_of_climate_change.csv